In [1]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100], dtype=int64))

In [3]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

#### Handle class imbalance

In [4]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)
np.unique(y_train_res, return_counts=True)

(array([0, 1]), array([619, 619], dtype=int64))

#### Track Experiments

In [5]:
models = [
    (
        "Logistic Regression", 
        {"C": 1, "solver": 'liblinear'},
        LogisticRegression(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest", 
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [6]:
reports = []

for model_name, params, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [7]:
reports

[{'0': {'precision': 0.9454545454545454,
   'recall': 0.9629629629629629,
   'f1-score': 0.9541284403669725,
   'support': 270.0},
  '1': {'precision': 0.6,
   'recall': 0.5,
   'f1-score': 0.5454545454545454,
   'support': 30.0},
  'accuracy': 0.9166666666666666,
  'macro avg': {'precision': 0.7727272727272727,
   'recall': 0.7314814814814814,
   'f1-score': 0.749791492910759,
   'support': 300.0},
  'weighted avg': {'precision': 0.9109090909090909,
   'recall': 0.9166666666666666,
   'f1-score': 0.91326105087573,
   'support': 300.0}},
 {'0': {'precision': 0.96415770609319,
   'recall': 0.9962962962962963,
   'f1-score': 0.9799635701275046,
   'support': 270.0},
  '1': {'precision': 0.9523809523809523,
   'recall': 0.6666666666666666,
   'f1-score': 0.7843137254901961,
   'support': 30.0},
  'accuracy': 0.9633333333333334,
  'macro avg': {'precision': 0.9582693292370712,
   'recall': 0.8314814814814815,
   'f1-score': 0.8821386478088503,
   'support': 300.0},
  'weighted avg': {'prec

In [8]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

In [20]:
#!pip install dagshub

In [21]:
#!pip install --upgrade httpx==0.25.0

In [22]:
#!pip install botocore==1.34.41

In [23]:
#!pip install jupyterlab==4.0.2

In [24]:
#!pip install dagshub --use-deprecated=legacy-resolver

In [25]:
#!pip list

In [26]:
#!pip install botocore==1.34.41

In [27]:
# dagshub setup
import dagshub
dagshub.init(repo_owner='ranesh88', repo_name='mlflow_demo', mlflow=True)

Initialized MLflow to track repo "ranesh88/mlflow_demo"

Repository ranesh88/mlflow_demo initialized!

In [28]:
# Ideally you will not require following 4 lines if you have started fresh and do not have any previous dagshub credentials on your computer
import os
os.environ['MLFLOW_TRACKING_USERNAME'] = 'ranesh88' # 'ranesh88'
os.environ['MLFLOW_TRACKING_PASSWORD'] = '1c7686a8179065bd764b7c5f644ce9d466c6fe79' # Password :: Prime@2024
os.environ['MLFLOW_TRACKING_URI'] = 'https://dagshub.com/ranesh88/mlflow_demo.mlflow' # https://dagshub.com/ranesh88/mlflow_demo.mlflow

# Initialize MLflow
mlflow.set_experiment("Anomaly Detection")
# mlflow.set_tracking_uri("http://localhost:5000")

for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):        
        mlflow.log_params(params)
        mlflow.log_metrics({
            'accuracy': report['accuracy'],
            'recall_class_1': report['1']['recall'],
            'recall_class_0': report['0']['recall'],
            'f1_score_macro': report['macro avg']['f1-score']
        })  
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")  

2024/09/07 16:43:10 INFO mlflow.tracking.fluent: Experiment with name 'Anomaly Detection' does not exist. Creating a new experiment.
2024/09/07 16:43:25 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/09/07 16:43:30 INFO mlflow.tracking._tracking_service.client: 🏃 View run Logistic Regression at: https://dagshub.com/ranesh88/mlflow_demo.mlflow/#/experiments/0/runs/4398e49c87f248099aee9669e0a507c3.
2024/09/07 16:43:30 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/ranesh88/mlflow_demo.mlflow/#/experiments/0.
2024/09/07 16:43:35 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/09/07 16:43:40 INFO mlflow.tracking._tracking_service.client: 🏃 View run Random Forest at: https://dagshub.com/